In [ ]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Output, Input, State
import dash_bootstrap_components as dbc
from dash_extensions import Download
from dash_extensions.snippets import send_data_frame
from dash.exceptions import PreventUpdate
import glob
import os
from pandas.tseries.offsets import BDay
from dash import dash_table


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy.stats as st
from scipy.stats import genextreme as gev
from scipy.stats import norm
from scipy.stats import exponweib
import warnings
warnings.filterwarnings("ignore")


data = pd.read_excel('data_KPI.xlsx')
del data['Unnamed: 0']

# inflation calculation
rates=[]
years = [*range(2012,2020)]

    
for j in range(8):                           
    rates.append(np.percentile(data['Value'].loc[data['Year'] == 2019],75)/np.percentile(data['Value'].loc[data['Year'] == years[j]],75))
    
for k in range(8):
    data.loc[data['Year'] == years[k], 'modded'] = round(data['Value']*rates[k]).astype(int)
    
#individual time series creation
series = []
grouped_df = data.groupby("ID")
for i in grouped_df:
    series.append(i[1])

#store players with 4+ years of data
reduced = []
for i in range(len(series)):
    if len(series[i])>=4:
        reduced.append(series[i])

#selected and others ID collection (Skill)
joid = []
roid = []
for i in range(len(reduced)):
    if reduced[i].Skill1.mean()>=80:
        joid.append(reduced[i].ID[:1].values)
    else:
        roid.append(reduced[i].ID[:1].values)
joid = [joid[i][0] for i in range(len(joid))]
roid = [roid[i][0] for i in range(len(roid))]
    
#creating datasets from the groups
selected_skill = data[data.ID.isin(joid)]
others_skill = data[data.ID.isin(roid)]

#selected and others ID collection (Value)
joid = []
roid = []
for i in range(len(reduced)):
    if reduced[i].Inlf_Value.mean()>=7:
        joid.append(reduced[i].ID[:1].values)
    else:
        roid.append(reduced[i].ID[:1].values)
joid = [joid[i][0] for i in range(len(joid))]
roid = [roid[i][0] for i in range(len(roid))]
    
#creating datasets from the groups
selected_value = data[data.ID.isin(joid)]
others_value = data[data.ID.isin(roid)]

# keeping young players
agedrop = (15,16,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45)
for i in agedrop:
    selected_skill = selected_skill.drop(selected_skill[selected_skill.Age == i].index)
    others_skill = others_skill.drop(others_skill[others_skill.Age == i].index)
    
    selected_value = selected_value.drop(selected_value[selected_value.Age == i].index)
    others_value = others_value.drop(others_value[others_value.Age == i].index)
    
    
def fitness_leag(val,s):
    if s in val.value_counts().nlargest(3):
        ydot2 = 1
    elif s in val.value_counts().nlargest(5):
        ydot2 = 0.75
    elif s in val.value_counts().nlargest(8):
        ydot2 = 0.5
    elif s in val.value_counts().nlargest(10):
        ydot2 = 0.25
    else:
        ydot2 = 0
    return ydot2

def player_cdf(val,s,features):
    x = np.sort(val)
    # get the cdf values
    b = np.arange(len(val)) / float(len(val))
    ydot = np.interp(s,x,b)
    print('Exact value:',ydot)

    # plotting
    plt.rc('font', size=10) #controls default text size
    plt.rc('axes', titlesize=10) #fontsize of the title
    plt.rc('axes', labelsize=20) #fontsize of the x and y labels
    plt.rc('xtick', labelsize=20) #fontsize of the x tick labels
    plt.rc('ytick', labelsize=20) #fontsize of the y tick labels
    plt.rc('legend', fontsize=16) #fontsize of the legend
    ax=plt.axes()
    ax.set_facecolor("whitesmoke")

    plt.xlabel('Player variable')
    plt.ylabel('Percentile')
    plt.hlines(y=0.75,xmax=max(val),xmin=min(val), color='g', label='Q3')
    plt.hlines(y=0.5,xmax=max(val),xmin=min(val), color='orange', label='Q2')
    plt.hlines(y=0.25,xmax=max(val),xmin=min(val), color='purple', label='Q1')
    plt.plot(x,b)
    plt.legend()
    plt.plot(s,ydot, 'ro')

    plt.show()
    return ydot

def evaluate(player_prob, pname, features, goal):
    final_prob = pd.DataFrame()
    for f in features:
        probs=[]
        probs.append(np.percentile(player_prob[f],75))
        final_prob[f]=probs
    prob = np.percentile(final_prob,75)
    print('The probability of',pname,'will achieve',goal,'goal is:',prob)
    return final_prob, prob

def skill_handbook(goal, pname, years, where):

    ages2 = where['Age'].loc[where['Name']==pname].values[:years]

    s1 = where['Skill1'].loc[where['Name']==pname].values[:years]
    s2 = where['Skill2'].loc[where['Name']==pname].values[:years]
    s3 = where['Skill3'].loc[where['Name']==pname].values[:years]
    s4 = where['Skill4'].loc[where['Name']==pname].values[:years]
    s5 = where['Skill5'].loc[where['Name']==pname].values[:years]
 
    iv = where['Inlf_Value'].loc[where['Name']==pname].values[:years]
    nat = where['National_teammates'].loc[where['Name']==pname].values[:years]
    leag = where['League'].loc[where['Name']==pname].values[:years]
    player_prob = pd.DataFrame()
    if goal=='Skill':
        player=[s2,s3,s4,s5,iv,nat,leag]
        features= ['Skill2','Skill3','Skill4','Skill5','Inlf_Value','National_teammates','League']
        for f in range(len(features)):
            j = 0
            a = []
            for i in ages2:
                if features[f]=='League':
                    print('Age:',i)
                    print('Examined:',features[f])
                    print('Player league:',player[f][j])
                    val = selected_skill[features[f]].loc[selected_skill['Age']==i]
                    ydot2 = fitness_leag(val, player[f][j])
                    print('Fitness:',ydot2)
                    a.append(ydot2)
                    j+=1
                    print('-----------------------------')
                else:
                    print('Age:',i)
                    print('Examined:',features[f])
                    print('Player value:',player[f][j])
                    val = selected_skill[features[f]].loc[selected_skill['Age']==i].values
                    ydot2 = player_cdf(val, player[f][j], features[f])
                    a.append(ydot2)
                    j+=1
                    print('-----------------------------')

            player_prob[features[f]] = a
            allscore = pd.DataFrame()
    elif goal=='Value':
        player=[s1,s2,s3,s4,s5,nat,leag]
        features= ['Skill1','Skill2','Skill3','Skill4','Skill5','National_teammates','League']  
        for f in range(len(features)):
            j = 0
            a = []
            for i in ages2:
                if features[f]=='League':
                    print('Age:',i)
                    print('Examined:',features[f])
                    print('Player league:',player[f][j])
                    val = selected_value[features[f]].loc[selected_value['Age']==i]
                    ydot2 = fitness_leag(val, player[f][j])
                    print('Fitness:',ydot2)
                    a.append(ydot2)
                    j+=1
                    print('-----------------------------')
                else:
                    print('Age:',i)
                    print('Examined:',features[f])
                    print('Player value:',player[f][j])
                    val = selected_value[features[f]].loc[selected_value['Age']==i].values
                    ydot2 = player_cdf(val, player[f][j], features[f])
                    a.append(ydot2)
                    j+=1
                    print('-----------------------------')

            player_prob[features[f]] = a
      
        
    final_prob, prob = evaluate(player_prob,pname, features, goal)
    return final_prob, features, prob

def player_selector(goal, age):
    if goal=='Skill':
        display(pd.concat([selected_skill[['Skill1','Name']].loc[(selected_skill['Age']==age) & (selected_skill['Skill1']<75)].sample(7),
        others_skill[['Skill1','Name']].loc[(others_skill['Age']==age) & (others_skill['Skill1']<75)].sample(7)]))
    elif goal=='Value':
        display(pd.concat([selected_value[['Inlf_Value','Name']].loc[(selected_value['Age']==age) & (selected_value['Inlf_Value']<6.8)].sample(7),
        others_value[['Inlf_Value','Name']].loc[(others_value['Age']==age) & (others_value['Inlf_Value']<6.8)].sample(7)]))

def career(goal, name):
    if goal=='Skill':
        print('Skill 1 goal: avg 80')
        print("")
        valus = []
        j=0
        for i in selected_skill[['Age','Skill1']].loc[selected_skill['Name']==name].values:
            valus.append(i[1])
            ag = i[0]
            print('Age:',ag,'Mean:',np.mean(valus))
        if len(valus)!=0:
            print("")
            print('Please enable: "selected_skill" dataset')
        valus = []
        j=0
        for i in others_skill[['Age','Skill1']].loc[others_skill['Name']==name].values:
            valus.append(i[1])
            ag = i[0]
            print('Age:',ag,'Mean:',np.mean(valus))
        if len(valus)!=0:
            print("")
            print('Please enable: "others_skill" dataset')
    elif goal=='Value':
        print('Value goal: avg 7 (~10M €)')
        print("")
        valus = []
        j=0
        for i in selected_value[['Age','Inlf_Value']].loc[selected_value['Name']==name].values:
            valus.append(i[1])
            ag = i[0]
            print('Age:',ag,'Mean:',np.mean(valus))
        if len(valus)!=0:
            print("")
            print('Please enable: "selected_value" dataset')
        valus = []
        j=0
        for i in others_value[['Age','Inlf_Value']].loc[others_value['Name']==name].values:
            valus.append(i[1])
            ag = i[0]
            print('Age:',ag,'Mean:',np.mean(valus))
        if len(valus)!=0:
            print("")
            print('Please enable: "others_value" dataset')
            
def visual(final_prob, features, pname, prob):
    mm = pd.DataFrame()
    np.mean(final_prob)

    for f in features:
        a = []
        a.append(np.mean(final_prob[f]))
        mm[f]=a
    display(final_prob)
    percentage = "{:.0%}". format(prob)
    fig = px.line_polar(mm, r = pd.Series(mm.loc[0,features].values),
                  theta=features,line_close=True,
                  title =print(pname,'fitness:', percentage))
    return fig

columns = ['Name','Age', 'Year', 'Skill1', 'Skill2', 'Skill3', 'Skill4', 'Skill5', 'League', 'National_teammates', 'Value', 'modded', 'Inlf_Value']
newplayer = pd.DataFrame(columns=columns)

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.SLATE])




tab1 =  html.Div([
    dbc.Row([
    dbc.Col([
    dbc.Card([
        dbc.Row([html.P('Select a goal:'), dcc.Dropdown(['Value', 'Skill'], id='goal-select')]),
        dbc.Row([html.P('Select a year:'), dcc.Dropdown(id='year-review', options=[{'label': x, 'value': int(x)} for x in data.Year.unique()])],  align="center"),
        dbc.Row([html.P('Select a player:'), dcc.Dropdown(id='player-stat')],  align="center"),
        html.Button('Reset', id='reset-button',n_clicks=0),
        dbc.Row([html.P('Select a player for analysis (not achieved the goal yet):'), dcc.Dropdown(id='player-select', multi=False, options=[{'label': x, 'value': x} for x in data.Name.unique()])]),
        dbc.Row([html.P('Select career length:'), dcc.Dropdown(id='age-select', multi=False)]),
        html.Button('Fitness', id='submit-val',n_clicks=0),
        html.Div(id='container-button-basic')
    ],style={'margin-left':'10px', 'margin-top':'8px', 'margin-right':'10px'}),
    ],width=2),
        
    dbc.Col([  
    dbc.Row([
        dbc.Col([dbc.Card([dcc.Graph(id='value-year-graph', figure={})], style={'margin-top':'8px', 'margin-bottom':'10px', 'margin-right':'5px'})]),
        dbc.Col([dbc.Card([dcc.Graph(id='league-pie', figure={})], style={'margin-top':'8px', 'margin-bottom':'10px', 'margin-left':'5px', 'margin-right':'10px'})])
    ],className="g-0"),        
    dbc.Row([
        dbc.Col([dbc.Card([dcc.Graph(id='player-pos-graph', figure={})], style={'margin-top':'10px', 'margin-bottom':'10px', 'margin-right':'5px'})]),
        dbc.Col([dbc.Card([dcc.Graph(id='value-hist', figure={})], style={'margin-top':'10px', 'margin-bottom':'10px', 'margin-left':'5px', 'margin-right':'10px'})])
    ],className="g-0"),   
    dbc.Row([
        dbc.Col([dbc.Card([dcc.Graph(id='my-graph', figure={})], style={'margin-top':'10px', 'margin-bottom':'8px', 'margin-right':'5px'})]),
        dbc.Col([dbc.Card([dcc.Graph(id='radar', figure={})], style={'margin-top':'10px', 'margin-bottom':'8px', 'margin-left':'5px', 'margin-right':'10px'})])
    ],className="g-0"),
    
    
    
        
    ], width = 10),
    ], className="g-0"),
]),
    
tab2 = html.Div([
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dcc.Input(id='Name', type='text', placeholder='Name', style={'width': '150px', 'margin-top':'5px', 'margin-left':'8px'}),
                dcc.Dropdown(options=[{'label': i, 'value': i} for i in range(17,27)],id='Age', placeholder='Age', style={'width': '100px', 'margin-top':'5px', 'margin-left':'4px'}),
                dcc.Dropdown(options=[{'label': i, 'value': i} for i in range(2012,2020)],id='Year', placeholder='Year', style={'width': '100px', 'margin-top':'5px', 'margin-left':'4px'}),
                dcc.Dropdown(options=[{'label': i, 'value': i} for i in data.League.unique()],id='league', placeholder='League', style={'width': '200px', 'margin-top':'5px', 'margin-left':'4px'}),
                dcc.Input(id='national', type='number', placeholder='National teammates', style={'width': '150px', 'margin-top':'5px', 'margin-left':'8px'}),
                dcc.Input(id='value', type='number', placeholder='Market value (€)', style={'width': '150px', 'margin-top':'5px', 'margin-left':'8px', 'margin-bottom':'5px'})       
            ])
        
        ], width={"order": "first"}, style={'margin-top':'8px'}),
        dbc.Col([
            dbc.Row([
                
                     dbc.Col([html.P('Skill 1'),
        dcc.Slider(1, 100, 1, id='Skill1', value=5, marks=None, tooltip={"placement": "bottom", "always_visible": True},vertical=True)
        ]),
        
        dbc.Col([html.P('Skill 2'),
        dcc.Slider(1, 100, 1, id='Skill2', value=5, marks=None, tooltip={"placement": "bottom", "always_visible": True},vertical=True)
        ]),
        
        dbc.Col([html.P('Skill 3'),
        dcc.Slider(1, 100, 1, id='Skill3', value=5, marks=None, tooltip={"placement": "bottom", "always_visible": True},vertical=True)
        ]),
        
        dbc.Col([html.P('Skill 4'),
        dcc.Slider(1, 100, 1, id='Skill4', value=5, marks=None, tooltip={"placement": "bottom", "always_visible": True},vertical=True)
        ]),
        
        dbc.Col([html.P('Skill 5'),
        dcc.Slider(1, 100, 1, id='Skill5', value=5, marks=None, tooltip={"placement": "bottom", "always_visible": True},vertical=True)
        ]),
            ]),
       
            
        ],style={'margin-top':'8px'}),
        
        
        dbc.Row([
            dbc.Col([html.Button("Add player", id="save", n_clicks=0),
            html.A(html.Button('Finish'),href='/')]),
            html.Div(id='container')
        ]),
        
        ])
]),

app.layout = html.Div([
dbc.Row([
    
    dbc.Col([
        dbc.Card([
    #dbc.CardImg(src="assets/sph.png"),
    dbc.CardBody(
            [
                html.H4("Soccer Player Handbook", className="card-title"),
                html.P("a simple way to boost your career", className="card-text", style={"font-style": "italic"}),
            ]
        ),
         ],style={"border-bottom": "1px solid #f2f2f2", "margin-bottom": "8px"}),
    ], width=11 ,className="g-0"),
     
    dbc.Col([
        dbc.Card([
    dbc.CardBody(
            [
                dbc.CardImg(src="assets/sph.png",style={"height" : "60px"}),
            ]
        ),
         ],style={"border-bottom": "1px solid #f2f2f2", "margin-bottom": "8px"}),
        
    ], width=1, className="g-0"),

    
    

    ],className="g-0"),
dbc.Tabs(
    [
        dbc.Tab(tab1, label="Handbook",  tabClassName="flex-grow-1 text-center", active_tab_style={"border-bottom": "5px solid #95eaea"}),
        dbc.Tab(tab2, label="Add player",  tabClassName="flex-grow-1 text-center", active_tab_style={"border-bottom": "5px solid #95eaea"}),           
    ]
),
])
#......................................................................................

# Add player
@app.callback(Output('container', 'children'),
                Input("save", "n_clicks"),
                State("Name", "value"),
                State('Age', 'value'),
State("Year", "value"),
State("Skill1", "value"),
State("Skill2", "value"),
State("Skill3", "value"),
State("Skill4", "value"),
State("Skill5", "value"),
State("league", "value"),
State("national", "value"),
State("value", "value"),
prevent_initial_call=True
)
def append(n_clicks, name, age, year, skill1, skill2, skill3, skill4, skill5, league, national, value):
    global newplayer
    global data
    rates=[]
    years = [*range(2012,2020)]
    for j in range(8):                           
        rates.append(np.percentile(data['Value'].loc[data['Year'] == 2019],75)/np.percentile(data['Value'].loc[data['Year'] == years[j]],75))
    modded = value*rates[year-2012]
    Inlf_Value = np.log10(modded)
    newplayer.loc[len(newplayer)+1]=[name, age, year, skill1, skill2, skill3, skill4, skill5, league, national, value, modded, Inlf_Value]
    return n_clicks,' row added'

# Handbook & handbook plot
@app.callback(
    Output('container-button-basic', 'children'),
    Output('radar', 'figure'),
    Input('submit-val', 'n_clicks'),
    State('goal-select', 'value'),
    State('player-select', 'value'),
    State('age-select', 'value')
)
def xyz(clicks, goal, pname, years):    
    global newplayer
    changed_ids = [p['prop_id'].split('.')[0] for p in dash.callback_context.triggered]
    button_pressed = 'submit-val' in changed_ids

    #if not button_pressed:
        #return ""
    
    if goal is None or pname is None or years is None:
        fig = px.line()
        fig.add_annotation(text="Select goal, player, length and click Fitness", showarrow=False)
        fig.update_annotations(font_size=22)
        fig.update_xaxes(showgrid=False, showticklabels=False)
        fig.update_yaxes(showgrid=False, showticklabels=False)
        fig.update_layout(paper_bgcolor="#32383E", font_color="white", plot_bgcolor="#32383E")
        a = ""
        return a, fig
    elif goal == 'Skill':
        global others_skill
        
        data = pd.concat([newplayer, others_skill])
        years = years
        final_prob, features, prob = skill_handbook(goal,pname,years,data)

        mm = pd.DataFrame()
        for f in features:
            a = []
            a.append(np.mean(final_prob[f]))
            mm[f]=a
        
        percentage = "{:.0%}". format(prob)
        fig = px.line_polar(mm, r = pd.Series(mm.loc[0,features].values),
                      theta=features,line_close=True,
                      title =print(pname,'fitness:', percentage))
        a = 'The probability of ',pname,' will achieve ',goal,' goal is: '"{:.2%}".format(prob)
        fig.update_layout(paper_bgcolor="#32383E", font_color="white", plot_bgcolor="#32383E")
        fig.update_polars(bgcolor="#32383E")
        fig.data[0].line.color = "yellow"
        return a, fig
            
    else:
        global others_value
        
        data = pd.concat([newplayer, others_value])
        years = years
        final_prob, features, prob = skill_handbook(goal,pname,years,data)
        
        mm = pd.DataFrame()
        for f in features:
            a = []
            a.append(np.mean(final_prob[f]))
            mm[f]=a
        
        percentage = "{:.0%}". format(prob)
        fig = px.line_polar(mm, r = pd.Series(mm.loc[0,features].values),
                      theta=features,line_close=True,
                      title =print(pname,'fitness:', percentage))
        a = 'The probability of ',pname,' will achieve ',goal,' goal is: '"{:.2%}".format(prob)
        fig.update_layout(paper_bgcolor="#32383E", font_color="white", plot_bgcolor="#32383E")
        fig.update_polars(bgcolor="#32383E")
        fig.data[0].line.color = "yellow"
        return a, fig
    
# Dashboard: PLayer selection
@app.callback(
    Output('player-stat', 'options'),
    Input('year-review', 'value')
)
def update_name_dropdwon(year):
    new = pd.concat([newplayer, data])
    return [{'label':x, 'value':x} for x in new[new['Year']==year].Name.unique()]

# Reset button
@app.callback(
    Output('player-stat', 'value'),
    Output('goal-select', 'value'),
    Output('year-review', 'value'),
    Input('reset-button', 'n_clicks')
)
def on_click(n_clicks):
    if n_clicks is None:
        raise PreventUpdate
    else:
        return None, None, None
    
# Auto increment
@app.callback(
    Output('Age', 'value'),
    Output('Year', 'value'),
    Input('save', 'n_clicks'),
    State('Age', 'value'),
    State("Year", "value")
)
def on_click(n_clicks, age, year):
    if n_clicks is None:
        raise PreventUpdate
    else:
        return age+1, year+1

# Analysis player selection  
@app.callback(
    Output('player-select', 'options'),
    Input('goal-select', 'value')
)
def update_name_dropdwon(goal):
    if goal == 'Value':
        hold = []
        for i in newplayer.Name.unique():
            if newplayer.Inlf_Value.loc[newplayer.Name==i].mean() < 7:
                hold.append(i)
        new = pd.concat([newplayer[newplayer['Name'].isin(hold)], others_value])
        #new = pd.concat([newplayer, others_value])
        return [{'label':x, 'value':x} for x in new[new['Age']<=24].Name.unique()]        
    else:
        hold = []
        for i in newplayer.Name.unique():
            if newplayer.Skill1.loc[newplayer.Name==i].mean() < 80:
                hold.append(i)
        new2 = pd.concat([newplayer[newplayer['Name'].isin(hold)], others_skill])
        #new2 = pd.concat([newplayer, others_skill])
        return [{'label':x, 'value':x} for x in new2[new2['Age']<=24].Name.unique()]

# Analysis career length
@app.callback(
    Output('age-select', 'options'),
    Input('player-select', 'value'),
    Input('goal-select', 'value')
)
def update_age_dropdown(selected, goal):
    if goal == 'Value':
        new = pd.concat([newplayer, others_value])
        return [{'label':x-(np.min(new[new['Name']==selected].Age.unique())-1), 'value':x-(np.min(new[new['Name']==selected].Age.unique())-1)} for x in new[new['Name']==selected].Age.unique()]        
    else:
        new2 = pd.concat([newplayer, others_skill])
        return [{'label':x-(np.min(new2[new2['Name']==selected].Age.unique())-1), 'value':x-(np.min(new2[new2['Name']==selected].Age.unique())-1)} for x in new2[new2['Name']==selected].Age.unique()]

# TOP 10 league pie
@app.callback(
    Output('league-pie', 'figure'),
    Input('year-review', 'value'),
    Input('goal-select', 'value'),
    Input('player-stat', 'value')
)
def update_graph(year, goal, player):
    global newplayer
    new = pd.concat([newplayer[newplayer.Year.isin([year])], data[data.Year.isin([year])]])
    if year is None:
        dff = data.groupby('League').count()
        dff = dff.nlargest(10, 'ID')
        fig = px.pie(dff, values='ID', names=dff.index, title='TOP 10 popular league - full history <br> <i style="font-size: 12px">The top 10 league with most player and its distribuiton in percentage</i>',width=600, height=400)
        fig.update_layout(paper_bgcolor="#32383E", font_color="white")
        fig.update_traces(hovertemplate='HELLO WORLD!')
        return fig
    elif player is None:
        dff = new[new.Year.isin([year])]
        dff = dff.groupby('League').count()
        dff = dff.nlargest(10, 'ID')
        fig = px.pie(dff, values='ID', names=dff.index, title='TOP 10 popular league - '+str(year)+'<br> <i style="font-size: 12px">The top 10 league with most player and its distribuiton in percentage</i>',width=600, height=400)
        fig.update_layout(paper_bgcolor="#32383E", font_color="white")
        return fig
    else:
        dff = new[new.Year.isin([year])]
        dff = dff.groupby('League').count()
        dff = dff.nlargest(10, 'ID')
        lig = new['League'].loc[new.Name==player].values[0]
        pull = dff.index.isin([lig])
        pull = np.where(pull == False, 0, 0.2)
        fig = px.pie(dff, values='ID', names=dff.index, title='TOP 10 popular league - '+str(year)+'<br> <i style="font-size: 12px">The top 10 league with most player and its distribuiton in percentage</i>',width=600, height=400)
        fig.update_layout(paper_bgcolor="#32383E", font_color="white")
        fig.update_traces(pull=pull, selector=dict(type='pie'))
        return fig

# Analysis player plot 
@app.callback(
    Output(component_id='my-graph', component_property='figure'),
    Input(component_id='player-select', component_property='value'),
    Input(component_id='goal-select', component_property='value')
)
def update_graph(player, goal):
    valq = selected_value.groupby('Age').modded.describe()['75%']
    skilq = selected_skill.groupby('Age').Skill1.describe()['75%']
    ages= valq.index.values
    modded = valq.values
    topq = 'Top Q3'
    skil = skilq.values
    upgr = pd.DataFrame({'Name':topq, 'Age': ages, 'modded': list(modded), 'Skill1': skil}, columns=['Name','Age','modded','Skill1'])
    if player is None:
        fig = px.line()
        fig.add_annotation(text="Select player & goal", showarrow=False)
        fig.update_annotations(font_size=28)
        fig.update_xaxes(showgrid=False, showticklabels=False)
        fig.update_yaxes(showgrid=False, showticklabels=False)
        fig.update_layout(paper_bgcolor="#32383E", font_color="white", plot_bgcolor="#32383E")
        return fig
    elif goal=='Value':
        new = pd.concat([newplayer, others_value, upgr])
        new = new[new.Name.isin([player, topq])]
        new = new.reset_index()
        maxi = new.Age.loc[new.Name==player].max()
        new.drop(new[(new.Name == topq)&(new.Age<maxi)].index, inplace=True)
        fig = px.line(data_frame=new, x='Age', y='modded', color='Name', labels={"modded": "Market value (€)"}, title='Examined player\'s Market value history & ideal future values')
        fig.update_traces(mode='lines+markers')
        fig.update_layout(paper_bgcolor="#32383E", font_color="white", plot_bgcolor="#32383E")
        fig.data[0].line.color = "yellow"
        fig.data[1].line.color = "#00ff00"
        return fig
    else:
        new = pd.concat([newplayer, others_skill, upgr])
        new = new[new.Name.isin([player, topq])]
        new = new.reset_index()
        maxi = new.Age.loc[new.Name==player].max()
        new.drop(new[(new.Name == topq)&(new.Age<maxi)].index, inplace=True)
        fig = px.line(data_frame=new, x='Age', y='Skill1', color='Name', title='Examined player\'s Skill history & ideal future values')
        fig.update_traces(mode='lines+markers')
        fig.update_layout(paper_bgcolor="#32383E", font_color="white", plot_bgcolor="#32383E")
        fig.data[0].line.color = "yellow"
        fig.data[1].line.color = "#00ff00"
        return fig

# Average market values
@app.callback(
    Output('value-year-graph', 'figure'),
    Input('year-review', 'value'),
    Input('goal-select', 'value'),
    Input('player-stat', 'value')
)
def update_side_graph(year, goal, player):
    global newplayer
    new = pd.concat([newplayer[newplayer.Year.isin([year])], data[data.Year.isin([year])]])
    dff = data.groupby('Year').mean()
    fig = px.bar(data_frame=dff, x=dff.index,y='modded', orientation='v', title='Average market values <br> <i style="font-size: 12px">The yearly mean market value between 2012 and 2019</i>',
                labels={"modded": "Market value (€)"}, width=600, height=400)
    fig.update_layout(paper_bgcolor="#32383E", font_color="white", plot_bgcolor="#32383E")
    return fig
        
# TOP 10 Player & position graph        
@app.callback(
    Output('player-pos-graph', 'figure'),
    Input('year-review', 'value'),
    Input('goal-select', 'value'),
    Input('player-stat', 'value')
)
def update_side_graph(year, goal, player):
    global newplayer
    new = pd.concat([newplayer[newplayer.Year.isin([year])], data[data.Year.isin([year])]])
    new.Skill1 = new.Skill1.astype(float)
    new['color'] = 0
    new['color'].loc[new['Name']==player] = 1
    if goal is None or year is None:
        fig = px.line(width=600, height=400)
        fig.add_annotation(text="Select goal, year, (player)", showarrow=False)
        fig.update_annotations(font_size=28)
        fig.update_xaxes(showgrid=False, showticklabels=False)
        fig.update_yaxes(showgrid=False, showticklabels=False)
        fig.update_layout(paper_bgcolor="#32383E", font_color="white", plot_bgcolor="#32383E")
        return fig
    elif goal=='Value':
        if player is None:
            dff = new.nlargest(10, 'modded')
            fig = px.bar(data_frame=dff, x='modded',y='Name', orientation='h', title='TOP 10 players with highest market value',
                         labels={"modded": "Market value (€)"}, width=600, height=400)
            fig.update_layout(yaxis={'categoryorder':'total ascending'})
            fig.update_layout(paper_bgcolor="#32383E", font_color="white", plot_bgcolor="#32383E")
            return fig
        else:
            plays = new.loc[new.Name==player]
            tops = new.nlargest(10, 'modded')
            if plays.Name.values[0] in tops.Name.values:
                fig = px.bar(data_frame=tops, x='modded',y='Name', orientation='h', title='Selected player market value position <br> <i style="font-size: 12px">The selected player compared to the top 10</i>',
                labels={"modded": "Market value (€)"}, width=600, height=400, color=tops['color'])
                fig.update_layout(yaxis={'categoryorder':'total ascending'})
                fig.update(layout_coloraxis_showscale=False)
                fig.update_layout(paper_bgcolor="#32383E", font_color="white", plot_bgcolor="#32383E")
                return fig
            else:
                viz = pd.concat([tops, plays])
                fig = px.bar(data_frame=viz, x='modded',y='Name', orientation='h', title='Selected player market value position <br> <i style="font-size: 12px">The selected player compared to the top 10</i>',
                labels={"modded": "Market value (€)"}, width=600, height=400, color=viz['color'])
                fig.update_layout(yaxis={'categoryorder':'total ascending'})
                fig.update(layout_coloraxis_showscale=False)
                fig.update_layout(paper_bgcolor="#32383E", font_color="white", plot_bgcolor="#32383E")
                return fig
    elif goal=='Skill':
        if player is None:
            dff = new.nlargest(10, 'Skill1')
            fig = px.bar(data_frame=dff, x='Skill1',y='Name', orientation='h', title='TOP 10 players with highest Skill value', width=600, height=400)
            fig.update_layout(yaxis={'categoryorder':'total ascending'})
            fig.update_layout(paper_bgcolor="#32383E", font_color="white", plot_bgcolor="#32383E")
            return fig
        else:
            plays = new.loc[new.Name==player]
            tops = new.nlargest(10, 'Skill1')
            if plays.Name.values[0] in tops.Name.values:
                fig = px.bar(data_frame=tops, x='Skill1',y='Name', orientation='h', title='Selected player Skill position <br> <i style="font-size: 12px">The selected player compared to the top 10</i>', 
                             width=600, height=400, color=tops['color'])
                fig.update_layout(yaxis={'categoryorder':'total ascending'})
                fig.update(layout_coloraxis_showscale=False)
                fig.update_layout(paper_bgcolor="#32383E", font_color="white", plot_bgcolor="#32383E")
                return fig
            else:
                viz = pd.concat([tops, plays])
                fig = px.bar(data_frame=viz, x='Skill1',y='Name', orientation='h', title='Selected player Skill position <br> <i style="font-size: 12px">The selected player compared to the top 10</i>',
                             width=600, height=400, color=viz['color'])
                fig.update_layout(yaxis={'categoryorder':'total ascending'})
                fig.update(layout_coloraxis_showscale=False)
                fig.update_layout(paper_bgcolor="#32383E", font_color="white", plot_bgcolor="#32383E")
                return fig
        
# Value histogram & Player position
@app.callback(
    Output('value-hist', 'figure'),
    Input('year-review', 'value'),
    Input('goal-select', 'value'),
    Input('player-stat', 'value')
)
def update_graph(year, goal, player):
    global newplayer
    new = pd.concat([newplayer[newplayer.Year.isin([year])], data[data.Year.isin([year])]])
    if goal is None or year is None:
        fig = px.line(width=600, height=400)
        fig.add_annotation(text="Select goal, year, (player)", showarrow=False)
        fig.update_annotations(font_size=28)
        fig.update_xaxes(showgrid=False, showticklabels=False)
        fig.update_yaxes(showgrid=False, showticklabels=False)
        fig.update_layout(paper_bgcolor="#32383E", font_color="white", plot_bgcolor="#32383E")
        return fig
    elif goal=='Value':
        dff = data[data.Year.isin([year])]
        if player is None:
            fig = px.histogram(data_frame=dff, x='Inlf_Value', title='Market value distribution', labels={"Inlf_Value": "Market value"}, width=600, height=400)
            fig.update_layout(paper_bgcolor="#32383E", font_color="white", plot_bgcolor="#32383E")
            return fig
        else:
            fig = px.histogram(data_frame=dff, x='Inlf_Value', title='Market value distribution <br> <i style="font-size: 12px">The selected player\'s position in the distribution</i>', labels={"Inlf_Value": "Market value"}, width=600, height=400)
            fig.update_layout(paper_bgcolor="#32383E", font_color="white", plot_bgcolor="#32383E")
            fig.add_vline(x=new['Inlf_Value'].loc[new['Name']==player].values[0], line_dash = 'dash', line_color = 'yellow')
            return fig
    elif goal=='Skill':
        dff = data[data.Year.isin([year])]
        if player is None:
            fig = px.histogram(data_frame=dff, x='Skill1', title='Skill distribution', width=600, height=400)
            fig.update_layout(paper_bgcolor="#32383E", font_color="white", plot_bgcolor="#32383E")
            return fig
        else:
            fig = px.histogram(data_frame=dff, x='Skill1', title='Skill distribution <br> <i style="font-size: 12px">The selected player\'s position in the distribution</i>', width=600, height=400)
            fig.update_layout(paper_bgcolor="#32383E", font_color="white", plot_bgcolor="#32383E")
            fig.add_vline(x=new['Skill1'].loc[new['Name']==player].values[0], line_dash = 'dash', line_color = 'yellow')
            return fig
        
        
if __name__ == '__main__':
    app.run_server(debug=False)